# **Procura de Problemas Éticos em Issues**

## **Imports necessários para o funcionamento:**

In [ ]:
%pip install pandas
%pip install nltk
%pip install ipywidgets


import pandas as pd
import re
import os
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import ipywidgets as widgets
from IPython.display import display
from collections import Counter

## **Fazendo a leitura dos dados:**

In [ ]:
issues = pd.read_csv("path_to_csv")
comentarios = pd.read_csv("path_to_csv")

# Concatenando o Titulo e a Descrição das issues
issues['Info'] = issues['TituloIssue'] + issues['DescricaoIssue']

# Convertendo todos os tipos para String
issues['Info'] = issues['Info'].astype(str).str.lower()
comentarios['Comentario'] = comentarios['Comentario'].astype(str).str.lower()

## **Tokenizando os Comentarios, Titulo e Descrição das Issues**

In [ ]:
# Definindo as StopWords
nltk.download('stopwords')
stopWords = set(stopwords.words('english'))

def removerStopWords(palavras):
    return [palavra for palavra in palavras if palavra not in stopWords and palavra.isalpha() or palavra in expressoes]

# Criando um tokenizer que considera expressões
expressoes = {'data governance', 'data protection','informed consent', 'lack of data', 'user data collection' 'business ethics','conflict of interest',
            'human agency', 'intellectual property','regulatory approaches', 'self-conception','professional ethics', 'work ethics','common goods','individual differences',
            'non-discrimination', 'non-maleficence','prevention of harm', 'quality of life', 'respect for human autonomy', 'retention and addiction', 'social justice',
            'speech issues', 'technical robustness','computer abuse', 'malicious use'}

expressaoRegular = r'\b(?:' + '|'.join(map(re.escape,expressoes)) + r')\b|\w+'
tokenizer = RegexpTokenizer(expressaoRegular)


# Tokenize dos comentarios das issues
comentariosToK = []
for i in range (comentarios['Comentario'].size):
    comentariosToK.append(removerStopWords(tokenizer.tokenize(comentarios.loc[i]['Comentario'])))

# Tokenize do campo Info
tituloDescricaoToK = []
for k in range(issues['Info'].size):
    tituloDescricaoToK.append(removerStopWords(tokenizer.tokenize(issues.loc[k]['Info'])))

## **Contando a ocorrência de problemas éticos:**

In [ ]:
from nltk.stem import RSLPStemmer
nltk.download('rslp')
radicalizador = RSLPStemmer()


# Definindo os EthicalIssues
EthicalIssues = ['bias', 'data', 'encryption', 'monetization', 'openness', 'privacy', 'authorship', 'autonomy', 'beneficence', 'commerce', 'compliance', 'confidentiality',
                 'context', 'dependability', 'fairness', 'oversight', 'responsibility', 'trust', 'trustworthiness', 'axiology', 'freedom', 'solidarity', 'utility',
                 'care', 'competence', 'access', 'accessibility', 'dignity', 'diversity', 'equality', 'equity', 'humanity', 'inclusiveness', 'inequality', 'justice', 
                 'participation', 'plurality', 'sustainability', 'unemployment', 'welfare', 'accountability', 'accuracy', 'anonymity', 'comprehensibility', 'consistency',
                 'contestability', 'explainability', 'explicability', 'integrity', 'interpretability', 'liability', 'reliability', 'safety', 'security', 'traceability', 'transparency',
                 'usability', 'data governance', 'data protection', 'informed consent', 'lack of data', 'user data collection', 'business ethics', 'conflict of interest', 
                 'human agency', 'intellectual property', 'regulatory approaches', 'professional ethics', 'individual differences', 'self-conception', 'work ethics', 
                 'common goods', 'non-discrimination', 'non-maleficence', 'prevention of harm', 'quality of life', 'respect for human autonomy', 'retention and addiction', 
                 'social justice', 'speech issues', 'technical robustness', 'computer abuse', 'malicious use']

# Criando o DataFrame que guardará as ocorrências dos comentários
ocorrenciasComents = pd.DataFrame()
linhaInicial = 0 * len(EthicalIssues)
palavrasAntes = "" * len(EthicalIssues)
palavrasDepois = "" * len(EthicalIssues)

ocorrenciasComents['EthicalIssues'] = EthicalIssues
ocorrenciasComents['Ocorrencias'] = linhaInicial
ocorrenciasComents["PalavrasAntes"] = palavrasAntes
ocorrenciasComents["PalavrasDepois"] = palavrasDepois
ocorrenciasComents = ocorrenciasComents.set_index('EthicalIssues')
ocorrenciasComents

# Criando o DataFrame que guardará as ocorrências do título e descrição
ocorrenciasTituloDescricao = pd.DataFrame()
linhaInicial = 0 * len(EthicalIssues)
palavrasAntes = "" * len(EthicalIssues)
palavrasDepois = "" * len(EthicalIssues)

ocorrenciasTituloDescricao['EthicalIssues'] = EthicalIssues
ocorrenciasTituloDescricao['Ocorrencias'] = linhaInicial
ocorrenciasTituloDescricao["PalavrasAntes"] = palavrasAntes
ocorrenciasTituloDescricao["PalavrasDepois"] = palavrasDepois
ocorrenciasTituloDescricao = ocorrenciasTituloDescricao.set_index('EthicalIssues')

EthicalIssues = [radicalizador.stem(palavra) for palavra in EthicalIssues]

In [ ]:

for i in range(len(comentariosToK)): 
    for j in range(len(comentariosToK[i])):
        
        if(radicalizador.stem(comentariosToK[i][j]) in EthicalIssues): # Acho que aqui seria o melhor momento para verificar (comentariosToK[i][j] seria radicalizada apenas para verificação)
            
            try:
                ocorrenciasComents.loc[comentariosToK[i][j], 'Ocorrencias'] += 1
            except:
                ocorrenciasComents.loc[comentariosToK[i][j]] = [1, "", ""]
                
            
            
            if((j - 1) >= 0):
                if(ocorrenciasComents.loc[comentariosToK[i][j],'PalavrasAntes'] == ''):
                    ocorrenciasComents.loc[comentariosToK[i][j], 'PalavrasAntes'] = comentariosToK[i][j - 1]
                else:
                    ocorrenciasComents.loc[comentariosToK[i][j], 'PalavrasAntes'] += ('\n' + comentariosToK[i][j - 1])
            if((j + 1) <= (len(comentariosToK[i]) - 1)):
                if(ocorrenciasComents.loc[comentariosToK[i][j], 'PalavrasDepois'] == ''):
                    ocorrenciasComents.loc[comentariosToK[i][j], 'PalavrasDepois'] = comentariosToK[i][j + 1]
                else:
                    ocorrenciasComents.loc[comentariosToK[i][j], 'PalavrasDepois'] += ('\n' + comentariosToK[i][j + 1])


for i in range(len(tituloDescricaoToK)):
    for j in range(len(tituloDescricaoToK[i])):
        
        if(radicalizador.stem(tituloDescricaoToK[i][j]) in EthicalIssues): # Acho que aqui seria o melhor momento para verificar (tituloDescricaoToK[i][j] seria radicalizada apenas para verificação)
            
            try:
                ocorrenciasTituloDescricao.loc[tituloDescricaoToK[i][j], 'Ocorrencias'] += 1
            except:
                ocorrenciasTituloDescricao.loc[tituloDescricaoToK[i][j]] = [1, "", ""]
            
            if((j - 1) >= 0):
                if(ocorrenciasTituloDescricao.loc[tituloDescricaoToK[i][j],'PalavrasAntes'] == ''):
                    ocorrenciasTituloDescricao.loc[tituloDescricaoToK[i][j], 'PalavrasAntes'] = tituloDescricaoToK[i][j - 1]
                else:
                    ocorrenciasTituloDescricao.loc[tituloDescricaoToK[i][j], 'PalavrasAntes'] += ('\n' + tituloDescricaoToK[i][j - 1])
            if((j + 1) <= (len(tituloDescricaoToK[i]) - 1)):
                if(ocorrenciasTituloDescricao.loc[tituloDescricaoToK[i][j], 'PalavrasDepois'] == ''):
                    ocorrenciasTituloDescricao.loc[tituloDescricaoToK[i][j], 'PalavrasDepois'] = tituloDescricaoToK[i][j + 1]
                else:
                    ocorrenciasTituloDescricao.loc[tituloDescricaoToK[i][j], 'PalavrasDepois'] += ('\n' + tituloDescricaoToK[i][j + 1])

## **Deixando o dataframe no formato correto:**

In [ ]:
ocorrenciasTituloDescricao = ocorrenciasTituloDescricao.reset_index()
ocorrenciasComents = ocorrenciasComents.reset_index()

ocorrenciasTituloDescricao['PalavrasAntes'] = ocorrenciasTituloDescricao['PalavrasAntes'].str.split('\n')
ocorrenciasTituloDescricao['PalavrasDepois'] = ocorrenciasTituloDescricao['PalavrasDepois'].str.split('\n')

ocorrenciasComents['PalavrasAntes'] = ocorrenciasComents['PalavrasAntes'].str.split('\n')
ocorrenciasComents['PalavrasDepois'] = ocorrenciasComents['PalavrasDepois'].str.split('\n')

ocorrenciasComents = ocorrenciasComents[ocorrenciasComents['Ocorrencias'] > 0]
ocorrenciasTituloDescricao = ocorrenciasTituloDescricao[ocorrenciasTituloDescricao['Ocorrencias'] > 0]

## **Criando uma tabela expansivel**

In [ ]:
# Função para criar a tabela com linhas expansíveis e contagem de palavras
def criarTabelaExpansivel(df):
    displayWidgets = []
    
    for _, linha in df.iterrows():
        # Conta as palavras em 'palavrasAntes' e 'palavrasDepois'
        antes_counts = Counter(linha['PalavrasAntes']).most_common()  # Ordena por contagem (decrescente)
        depois_counts = Counter(linha['PalavrasDepois']).most_common()  # Ordena por contagem (decrescente)
        
        # Cria uma linha com os dados principais
        linha_widget = widgets.HBox([
            widgets.Label(value=str(linha["EthicalIssues"]), layout=widgets.Layout(width="120px")),
            widgets.Label(value=str(linha["Ocorrencias"]), layout=widgets.Layout(width="60px")),
            widgets.Label(value="Clique para expandir", layout=widgets.Layout(width="200px"))
        ])
        
        # Cria o conteúdo expansível com `palavrasAntes` e `palavrasDepois` e suas contagens
        widgetDetalhes = widgets.VBox([
            widgets.Label(value=f"Palavras Antes: {', '.join([f'{word} ({count})' for word, count in antes_counts])}"),
            widgets.Label(value=f"Palavras Depois: {', '.join([f'{word} ({count})' for word, count in depois_counts])}")
        ])
        
        # Torna o conteúdo expansível
        expansao = widgets.Accordion(children=[widgetDetalhes])
        expansao.set_title(0, f"Detalhes de '{linha['EthicalIssues']}'")
        
        # Agrupa a linha principal e o conteúdo expansível
        linha_box = widgets.VBox([linha_widget, expansao])
        displayWidgets.append(linha_box)
    
    # Exibe a tabela com todas as linhas e detalhes
    display(widgets.VBox(displayWidgets))

## **Exibindo os resultados:**

In [ ]:
# Exibe a tabela expansível
criarTabelaExpansivel(ocorrenciasComents)

In [ ]:
# Exibe a tabela expansível
criarTabelaExpansivel(ocorrenciasTituloDescricao)

# **Exportando as Tabelas**

In [ ]:
# exportando como html

def exportar_para_html(df, filepath):
    html = """<html>
    <head>
        <meta charset="UTF-8">
        <style>
            table {width: 100%%; border-collapse: collapse;}
            th, td {border: 1px solid black; padding: 8px; text-align: left;}
            th {background-color: #f2f2f2;}
            .detalhes {display: none;}
        </style>
        <script>
            function toggleDetalhes(id) {
                var elemento = document.getElementById(id);
                elemento.style.display = (elemento.style.display === "none") ? "block" : "none";
            }
        </script>
    </head>
    <body>
        <h2>Tabela Expansível</h2>
        <table>
            <tr>
                <th>Ethical Issues</th>
                <th>Ocorrências</th>
                <th>Detalhes</th>
            </tr>
    """

    for i, (_, linha) in enumerate(df.iterrows()):
        antes_counts = Counter(linha["PalavrasAntes"]).most_common()
        depois_counts = Counter(linha["PalavrasDepois"]).most_common()

        detalhes = f"""<p><strong>Palavras Antes:</strong> {', '.join([f'{word} ({count})' for word, count in antes_counts])}</p>
                       <p><strong>Palavras Depois:</strong> {', '.join([f'{word} ({count})' for word, count in depois_counts])}</p>"""

        html += f"""
            <tr>
                <td>{linha["EthicalIssues"]}</td>
                <td>{linha["Ocorrencias"]}</td>
                <td><button onclick="toggleDetalhes('detalhes{i}')">Expandir</button></td>
            </tr>
            <tr id="detalhes{i}" class="detalhes">
                <td colspan="3">{detalhes}</td>
            </tr>
        """

    html += "</table></body></html>"

    with open(filepath, "w", encoding="utf-8") as f:
        f.write(html)


In [ ]:
exportar_para_html(ocorrenciasComents, "path_to_html")
exportar_para_html(ocorrenciasTituloDescricao, "path_to_html")